In [ ]:
!pip3 install torch

In [6]:
import torch
import numpy as np
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim

Load CIFAR-10 dataset

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalizing pixel values to be between 0 and 1

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Differential Equation for the Liquid State

In [3]:
class LiquidTimeStep(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LiquidTimeStep, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.W_in = nn.Linear(input_size, hidden_size)
        self.W_h = nn.Linear(hidden_size, hidden_size)
        self.tau = nn.Parameter(torch.ones(hidden_size))

    def forward(self, x, h):
        dx = torch.tanh(self.W_in(x) + self.W_h(h))
        h_new = h + (dx - h) / self.tau
        return h_new

LNN

In [9]:
class LNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        self.W_in = np.random.randn(input_size, hidden_size)
        self.W_hid = np.random.randn(hidden_size, hidden_size)
        self.W_out = np.random.randn(hidden_size, output_size)
        self.bias_hid = np.zeros(hidden_size)
        self.bias_out = np.zeros(output_size)
        self.time_constant = 0.1

    def forward(self, x):
        hidden_state = np.zeros(self.W_hid.shape[1])
        outputs = []

        for t in range(len(x)):
            hidden_state = (1 - self.time_constant) * hidden_state + self.time_constant * np.tanh(np.dot(x[t], self.W_in) + np.dot(hidden_state, self.W_hid) + self.bias_hid)

            output = np.dot(hidden_state, self.W_out) + self.bias_out

            exp_output = np.exp(output - np.max(output))  # for numerical stability
            softmax_output = exp_output / np.sum(exp_output)

            outputs.append(softmax_output)

        return np.array(outputs)

Example Usage

In [ ]:
x_train_flattened = x_train.reshape(x_train.shape[0], -1)  # Shape: (N, 3072)
x_test_flattened = x_test.reshape(x_test.shape[0], -1)    # Shape: (N, 3072)

input_size = 32 * 32 * 3
hidden_size = 20
output_size = 10
net = LNN(input_size, hidden_size, output_size)

predictions = net.forward(x_train_flattened[:10])

print(predictions)